In [2]:
import pandas as pd
import tensorflow as tf
import os
import numpy as np
import numpy as np
#from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Flatten, Dense,Conv2D,MaxPool2D,Activation,Input
from tensorflow.keras.models import Model
from tensorflow.keras import applications
from sklearn.model_selection import train_test_split
##Time taken to load the images
import time
import datetime
tf.keras.backend.clear_session()

S:\ana\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
S:\ana\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
S:\ana\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
S:\ana\lib\site-packages\tensorflow\python\framework\dtypes.py:529: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future 

### Model 1

In [13]:
tf.keras.backend.clear_session()
model = applications.VGG16(input_shape=(64,64,3),include_top=False, weights='imagenet')

In [14]:
Conv1 = Conv2D(filters=32,kernel_size=(3,3),strides=(1,1),padding='same',data_format='channels_last',
              activation='relu',kernel_initializer=tf.keras.initializers.he_normal(seed=0),name='Conv1')(model.output)
Pool1 = MaxPool2D(pool_size=(2,2),strides=(2,2),padding='valid',data_format='channels_last',name='Pool1')(Conv1)

flatten = Flatten(data_format='channels_last',name='Flatten')(Pool1)
FC1 = Dense(units=20,activation='relu',kernel_initializer=tf.keras.initializers.glorot_normal(seed=32),name='FC1')(flatten)
FC2 = Dense(units=20,activation='relu',kernel_initializer=tf.keras.initializers.glorot_normal(seed=33),name='FC2')(FC1)
Out = Dense(units=16,activation='softmax',kernel_initializer=tf.keras.initializers.glorot_normal(seed=3),name='Output')(FC2)
model = Model(inputs=model.input,outputs=Out)

In [15]:
for layer in model.layers[:19]:
    layer.trainable=False

In [16]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 64, 64, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, 64, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 64, 64, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 32, 32, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 32, 32, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 32, 32, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 16, 16, 128)       0         
__________

In [17]:
tf.keras.utils.plot_model(
    model,
    to_file='model1_architecture.png',
    show_shapes=True)

In [6]:
df=pd.read_csv("labels_final.csv",dtype=str)
ImageFlow = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255.,validation_split=0.25)

In [8]:
df = df.sample(frac=1).reset_index(drop=True)

In [10]:
train_generator=ImageFlow.flow_from_dataframe(
dataframe=df,
directory="./data_final/",
x_col="path",
y_col="label",
batch_size=32,
seed=42,
subset="training",
shuffle=False,
class_mode="categorical",
target_size=(64,64))

validation_generator=ImageFlow.flow_from_dataframe(
dataframe=df,
directory="./data_final/",
x_col="path",
y_col="label",
batch_size=32,
subset="validation",
seed=42,
shuffle=False,
class_mode="categorical",
target_size=(64,64))

Found 36000 validated image filenames belonging to 16 classes.
Found 12000 validated image filenames belonging to 16 classes.


In [11]:
STEP_SIZE_TRAIN=36000//32
STEP_SIZE_VALID=12000//32

In [12]:
log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, write_graph=True,write_grads=True)
model.compile(optimizer='rmsprop',loss='categorical_crossentropy', metrics=['accuracy'])

In [13]:
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=validation_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=19,callbacks=[tensorboard_callback]
)

Instructions for updating:
Use tf.cast instead.
Epoch 1/19
1125/1125 [==============================] - 2839s 3s/step - loss: 2.0778 - acc: 0.3517 - val_loss: 1.8806 - val_acc: 0.4114
Epoch 2/19
1125/1125 [==============================] - 1275s 1s/step - loss: 1.8036 - acc: 0.4391 - val_loss: 1.7431 - val_acc: 0.4635
Epoch 3/19
1125/1125 [==============================] - 1152s 1s/step - loss: 1.7139 - acc: 0.4670 - val_loss: 1.7024 - val_acc: 0.4750
Epoch 4/19
1125/1125 [==============================] - 1153s 1s/step - loss: 1.6489 - acc: 0.4890 - val_loss: 1.6904 - val_acc: 0.4815
Epoch 5/19
1125/1125 [==============================] - 1152s 1s/step - loss: 1.6015 - acc: 0.5027 - val_loss: 1.6760 - val_acc: 0.4891
Epoch 6/19
1125/1125 [==============================] - 1152s 1s/step - loss: 1.5644 - acc: 0.5125 - val_loss: 1.6310 - val_acc: 0.4999
Epoch 7/19
1125/1125 [==============================] - 1158s 1s/step - loss: 1.5345 - acc: 0.5234 - val_loss: 1.6025 - val_acc: 0.5102


In [14]:
model.save_weights('model1_weights.h5')

## Observations:

![title](model1_hist.png)

1. training and validation accuracy reaches about 0.5
2. As there are less parammeters to train, there is no overfitting
3. bias vary from -0.4 to 0.3 and weights vary from -4 to 1
4. Model accuracy plkeatues at 10th epoch

### Model 2

In [10]:
tf.keras.backend.clear_session()
model = applications.VGG16(input_shape=(64,64,3),include_top=False, weights='imagenet')

In [11]:
Conv1 = Conv2D(filters=32,kernel_size=(2,2),strides=(1,1),padding="valid",data_format='channels_last',
              activation='relu',kernel_initializer=tf.keras.initializers.he_normal(seed=0),name='Conv1')(model.output)

Conv2 = Conv2D(filters=20,kernel_size=(1,1),strides=(1,1),padding='valid',data_format='channels_last',
              activation='relu',kernel_initializer=tf.keras.initializers.he_normal(seed=0),name='Conv2')(Conv1)

Conv3 = Conv2D(filters=16,kernel_size=(1,1),strides=(1,1),padding='valid',data_format='channels_last',
              activation='softmax',kernel_initializer=tf.keras.initializers.he_normal(seed=0),name='Output')(Conv2)

flatten = Flatten(data_format='channels_last',name='Flatten')(Conv3)

finalmodel = Model(inputs=model.input,outputs=flatten)

#finalmodel = Model(inputs=model.input,outputs=Out)

In [12]:
for layer in finalmodel.layers[:19]:
    layer.trainable=False

In [13]:
finalmodel.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 64, 64, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, 64, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 64, 64, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 32, 32, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 32, 32, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 32, 32, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 16, 16, 128)       0         
__________

In [19]:
tf.keras.utils.plot_model(
    model,
    to_file='model2_architecture.png',
    show_shapes=True)

In [7]:
df=pd.read_csv("labels_final.csv",dtype=str)
ImageFlow = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255.,validation_split=0.25)

In [8]:
df = df.sample(frac=1).reset_index(drop=True)

In [9]:
train_generator=ImageFlow.flow_from_dataframe(
dataframe=df,
directory="./data_final/",
x_col="path",
y_col="label",
batch_size=32,
seed=42,
subset="training",
shuffle=False,
class_mode="categorical",
target_size=(64,64))

validation_generator=ImageFlow.flow_from_dataframe(
dataframe=df,
directory="./data_final/",
x_col="path",
y_col="label",
batch_size=32,
subset="validation",
seed=42,
shuffle=False,
class_mode="categorical",
target_size=(64,64))

Found 36000 validated image filenames belonging to 16 classes.
Found 12000 validated image filenames belonging to 16 classes.


In [17]:
STEP_SIZE_TRAIN=36000//32
STEP_SIZE_VALID=12000//32

In [18]:
log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, write_graph=True,write_grads=True)
finalmodel.compile(optimizer='rmsprop',loss='categorical_crossentropy', metrics=['accuracy'])

In [19]:
finalmodel.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=validation_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=20,callbacks=[tensorboard_callback]
)

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Epoch 1/20
1125/1125 [==============================] - 2064s 2s/step - loss: 2.1241 - acc: 0.3344 - val_loss: 1.9036 - val_acc: 0.4110
Epoch 2/20
1125/1125 [==============================] - 1160s 1s/step - loss: 1.8136 - acc: 0.4317 - val_loss: 1.7535 - val_acc: 0.4472
Epoch 3/20
1125/1125 [==============================] - 1150s 1s/step - loss: 1.7146 - acc: 0.4632 - val_loss: 1.7531 - val_acc: 0.4573
Epoch 4/20
1125/1125 [==============================] - 1153s 1s/step - loss: 1.6613 - acc: 0.4819 - val_loss: 1.6891 - val_acc: 0.4778
Epoch 5/20
1125/1125 [==============================] - 1156s 1s/step - loss: 1.6154 - acc: 0.4963 - val_loss: 1.6264 - val_acc: 0.4921
Epoch 6/20
1125/1125 [==============================] - 1149s 1s/step - loss: 1.5840 - acc: 0.5045 - val_loss: 1.5976 - val_acc: 0.5057
Epoch 7/20
1125/1125 [==============================] - 11

## Observations

![title](model2_hist.png)

1. train and validation accuracy remain reach about 52%
2. Since there is few parameters to train, no overfiting is obsereved
3. bias varies from -0.5 to 0.3 and weights vary from -4 to 1

### Model 3

In [3]:
tf.keras.backend.clear_session()
model = applications.VGG16(input_shape=(64,64,3),include_top=False, weights='imagenet')
Conv1 = Conv2D(filters=32,kernel_size=(2,2),strides=(1,1),padding="valid",data_format='channels_last',
              activation='relu',kernel_initializer=tf.keras.initializers.he_normal(seed=0),name='Conv1')(model.output)

Conv2 = Conv2D(filters=20,kernel_size=(1,1),strides=(1,1),padding='valid',data_format='channels_last',
              activation='relu',kernel_initializer=tf.keras.initializers.he_normal(seed=0),name='Conv2')(Conv1)

Conv3 = Conv2D(filters=16,kernel_size=(1,1),strides=(1,1),padding='valid',data_format='channels_last',
              activation='softmax',kernel_initializer=tf.keras.initializers.he_normal(seed=0),name='Output')(Conv2)

flatten = Flatten(data_format='channels_last',name='Flatten')(Conv3)

finalmodel = Model(inputs=model.input,outputs=flatten)

#finalmodel = Model(inputs=model.input,outputs=Out)

Instructions for updating:
Colocations handled automatically by placer.


In [4]:
for layer in finalmodel.layers[:13]:
    layer.trainable=False

In [5]:
finalmodel.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 64, 64, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, 64, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 64, 64, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 32, 32, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 32, 32, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 32, 32, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 16, 16, 128)       0         
__________

In [22]:
tf.keras.utils.plot_model(
    model,
    to_file='model3_architecture.png',
    show_shapes=True)

In [6]:
STEP_SIZE_TRAIN=36000//32
STEP_SIZE_VALID=12000//32
log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, write_graph=True,write_grads=True)
finalmodel.compile(optimizer='sgd',loss='categorical_crossentropy', metrics=['accuracy'])

In [10]:
finalmodel.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=validation_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=20,callbacks=[tensorboard_callback]
)

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Epoch 1/20
1125/1125 [==============================] - 2372s 2s/step - loss: 2.0918 - acc: 0.3498 - val_loss: 1.7858 - val_acc: 0.4608
Epoch 2/20
1125/1125 [==============================] - 1919s 2s/step - loss: 1.6142 - acc: 0.4958 - val_loss: 1.4858 - val_acc: 0.5478
Epoch 3/20
1125/1125 [==============================] - 1924s 2s/step - loss: 1.4207 - acc: 0.5574 - val_loss: 1.4327 - val_acc: 0.5677
Epoch 4/20
1125/1125 [==============================] - 1928s 2s/step - loss: 1.2884 - acc: 0.5998 - val_loss: 1.3365 - val_acc: 0.5910
Epoch 5/20
1125/1125 [==============================] - 1927s 2s/step - loss: 1.1764 - acc: 0.6341 - val_loss: 1.2942 - val_acc: 0.6086
Epoch 6/20
1125/1125 [==============================] - 1925s 2s/step - loss: 1.0766 - acc: 0.6616 - val_loss: 1.3003 - val_acc: 0.6204
Epoch 7/20
1125/1125 [==============================] - 19

In [11]:
model.save_weights('model3_weights.h5')

## Observation:

![title](model2_hist.png)

1. max training accuracy = 0.96 and max validation accuracy =0.63
2. Since 6 layers of vgg 16 were unfroze, there are more parameters to train which leads to overfitting
3. Overfitting starts from 9th epoch
4. bias varies from -0.25 to 0.19 and weights vary from -1.8 to 1.05
5. No cases of exploding/vanishing gradient problem